In [1]:
import multiprocessing as mp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import math
from tqdm import tqdm
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import datetime
from sklearn.preprocessing import quantile_transform
pd.options.mode.chained_assignment = None  # default='warn'
import gc

In [2]:
%%time
inter_df = pd.read_csv('floor_info_maindata_inter_baseline_2.csv')

Wall time: 9.83 s


In [3]:
inter_df.shape

(6880841, 16)

In [4]:
inter_df.head()

,관리_건축물대장_PK,건물_명,층_구분,층_번호,층_번호_명,구조,기타_구조,주_용도,기타_용도,면적(㎡),주_부속_구분,면적_제외_여부,생성_일자,기준년월,지번주소,addr_split_len
0,11440-3279202,중동빌라,지상,1,1층,철근콘크리트구조,철근콘크리트구조,다세대주택,다세대주택,57.62,주건축물,NaN,20141109,201412,서울특별시 마포구 중동 81-29,4
1,11440-3279202,중동빌라,지상,2,2층,철근콘크리트구조,철근콘크리트구조,다세대주택,다세대주택,67.20,주건축물,NaN,20141109,201412,서울특별시 마포구 중동 81-29,4
2,11440-3279202,중동빌라,지상,3,3층,철근콘크리트구조,철근콘크리트구조,다세대주택,다세대주택,67.20,주건축물,NaN,20141109,201412,서울특별시 마포구 중동 81-29,4
3,11440-3279202,중동빌라,지상,4,4층,철근콘크리트구조,철근콘크리트구조,다세대주택,다세대주택,59.68,주건축물,NaN,20141109,201412,서울특별시 마포구 중동 81-29,4
4,11440-3279202,중동빌라,옥탑,1,옥탑 1층,철근콘크리트구조,철근콘크리트구조,다세대주택,계단실(연면적제외),7.68,주건축물,NaN,20141109,201412,서울특별시 마포구 중동 81-29,4


In [6]:
inter_df['주_용도'].value_counts()

다세대주택               3077733
아파트                 2518366
연립주택                 278731
부대시설                 263734
오피스텔                 213753
사무소                   79058
소매점                   59215
단독주택                  48828
기타일반업무시설              42251
기타제1종근린생활시설           41779
주차장                   36028
기타제2종근린생활시설           24395
생활편익시설                21767
다가구주택                 20621
일반음식점                 20565
복리시설                  11205
기타판매시설                 8276
학원                     8138
기타사무소                  7867
상점                     6520
기타제1종근생활시설             5909
의원                     5638
휴게음식점                  5540
기타근린생활시설               4763
대피소                    3647
유치원                    3571
기타제2종근생활시설             3092
노인복지시설                 2652
관광호텔                   2107
제조업소                   2053
창고                     1867
독서실                    1634
부동산중개사무소               1626
영유아보육시설                1451
기타교육연구시설               1419
부동산중개업소             

In [10]:
inter_df['기타_용도'].value_counts()[:100]

아파트                          1587535
다세대주택(2세대)                    653388
다세대주택                         460085
계단실                           244509
다세대주택(1세대)                    240276
계단실(연면적제외)                    216981
주차장                           203933
공동주택(아파트)                     151118
다세대주택(3세대)                    116301
연립주택                          104405
주택                             98456
아파트(4세대)                       85909
아파트(2세대)                       83967
다세대주택 (2세대)                    72667
주택(2세대)                        60714
오피스텔                           50942
다세대주택(4세대)                     47816
아파트(3세대)                       41800
다세대주택(2)                       36821
지하주차장                          35842
업무시설(오피스텔)                     32133
연면적제외                          32073
다세대주택 (1세대)                    32020
근린생활시설                         31536
경비실                            31155
제2종근린생활시설(사무소)                 31095
주택(1세대)                        27304
아

In [7]:
inter_df['주_부속_구분'].value_counts()

주건축물     6721119
부속건축물     159241
Name: 주_부속_구분, dtype: int64

In [8]:
sub_df = inter_df[inter_df['주_부속_구분'] != '주건축물']
sub_df['주_용도'].value_counts()

부대시설           81912
아파트            58285
복리시설            8213
주차장             1885
연립주택            1541
단독주택            1031
다세대주택            961
생활편익시설           753
기타제2종근린생활시설      533
오피스텔             456
기타사무소            438
기타제1종근린생활시설      300
노인복지시설           293
공동주택             252
대피소              198
창고               169
영유아보육시설          160
기타노유자시설          151
기타판매시설           142
독서실              140
기타일반업무시설         137
차고               128
소매점              105
공중화장실            100
상점                78
기타아동관련시설          77
사무소               73
기타근린생활시설          61
교회                60
일반공장              58
사회복지시설            53
일반음식점             44
기타제2종근생활시설        44
기타창고시설            40
다가구주택             39
기타 운동시설           35
기타교육연구시설          34
기타공장              34
기타자동차관련시설         31
마을공동시설            31
유치원               28
창고시설              22
기타제1종근생활시설        21
체력단련장             19
학원                16
기숙사               16
근린생활시설            16
호텔           